In [ ]:
# Import necessary libraries
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

from model.sasrec import SASRecModel
from trainers import Trainer
from utils import EarlyStopping, check_path, set_seed, set_logger
from dataset import get_seq_dic, get_dataloder, get_rating_matrix

# Set up arguments
class Args:
    data_dir = "./data/"
    output_dir = "output/"
    data_name = "input_search_augmented_final"
    do_eval = False
    load_model = None
    train_name = "test_model"
    num_items = 10
    num_users = 10
    lr = 0.001
    batch_size = 256
    epochs = 10
    no_cuda = False
    log_freq = 1
    patience = 2
    num_workers = 0  # Set num_workers to 0 to avoid BrokenPipeError on Windows
    seed = 42
    weight_decay = 0.0
    adam_beta1 = 0.9
    adam_beta2 = 0.999
    gpu_id = "0"
    variance = 5
    model_type = 'bert4rec'
#     model_type = 'sasrec_model'
    max_seq_length = 50
    hidden_size = 256
    num_hidden_layers = 2
    hidden_act = "gelu"
    num_attention_heads = 2
    attention_probs_dropout_prob = 0.5
    hidden_dropout_prob = 0.5
    initializer_range = 0.02

args = Args()

In [ ]:
df = pd.read_csv("/data/log-data-2024/SASRec/BSARec/src/data/input_search_sample_matching_final.txt")
samp = list(pd.read_csv("/data/log-data-2024/8man_sample_new.csv")["treatment1"])

df["live"] = [1 if i in samp else 0 for i in tqdm(df["device_id"])]
df = df[df["live"] == 1].reset_index(drop = True)

print(list(df["device_id"]) == samp)
input_data = df["sequence"]

input_data = [" ".join(i.split()[1:]) for i in input_data]
    
input_ids = []
for line in input_data:
    items = list(map(int, line.strip().split()))
    pad_len = args.max_seq_length - len(items)
    input_ids.append([0] * pad_len + items)
    
# get attention weights
model = torch.load("bert4rec.pt")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
input_ids = torch.tensor(input_ids).to(device)
model.eval()

with torch.no_grad():
    extended_attention_mask = model.get_attention_mask(input_ids)
    sequence_emb = model.add_position_embedding(input_ids)
    item_encoded_layers = model.item_encoder(sequence_emb, extended_attention_mask, output_all_encoded_layers=True)
    attention_weights = item_encoded_layers[-1]  # Extract the attention weights from the last layer

mean_attention_weights = torch.mean(attention_weights, dim=-1)
print(mean_attention_weights.size())
attention_probs = torch.nn.functional.softmax(mean_attention_weights, dim=-1)

attention_probs = attention_probs.tolist()
input_ids = input_ids.tolist()

ap = []
ii = []
for a, i in enumerate(input_ids):
    while 0 in i:
        i.remove(0)
    ap.append(attention_probs[a][len(i) * (-1):])
    ii.append(i)

In [ ]:
import pickle

focus = []    
for a, i in enumerate(ap):
    first_max_index = np.argmax(i)
    i[first_max_index] = -np.inf
    second_max_index = np.argmax(i)
    focus.append([ii[a][first_max_index], ii[a][second_max_index]])
    
with open(file='/data/log-data-2024/SASRec/BSARec/src/SASRec_prediction_attention.pickle', mode='wb') as f:
    pickle.dump(focus, f)